In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [0]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle


In [0]:
!pip install kaggle

In [0]:
!kaggle competitions download jigsaw-multilingual-toxic-comment-classification
#!kaggle datasets download -d datasnaek/youtube-new

 80% 24.0M/29.8M [00:00<00:00, 65.1MB/s]
100% 29.8M/29.8M [00:00<00:00, 85.8MB/s]
 85% 68.0M/79.6M [00:00<00:00, 108MB/s]
100% 79.6M/79.6M [00:00<00:00, 109MB/s]
  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 44.7MB/s]
  0% 0.00/3.44M [00:00<?, ?B/s]
100% 3.44M/3.44M [00:00<00:00, 56.8MB/s]
  0% 0.00/612k [00:00<?, ?B/s]
100% 612k/612k [00:00<00:00, 41.0MB/s]
 99% 644M/650M [00:08<00:00, 45.0MB/s]
100% 650M/650M [00:08<00:00, 84.1MB/s]
 99% 290M/292M [00:03<00:00, 60.5MB/s]
100% 292M/292M [00:03<00:00, 90.1MB/s]
 91% 34.0M/37.3M [00:00<00:00, 73.6MB/s]
100% 37.3M/37.3M [00:00<00:00, 107MB/s] 
 40% 5.00M/12.4M [00:00<00:00, 34.1MB/s]
100% 12.4M/12.4M [00:00<00:00, 60.7MB/s]


In [0]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [0]:
!pip install transformers
!pip install tensorflow==2.1.0

     |████████████████████████████████| 645kB 2.5MB/s 
     |████████████████████████████████| 3.8MB 6.5MB/s 
     |████████████████████████████████| 1.1MB 32.9MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=97c5ff02246df69b93be98b666a59084f779e5b43f875934ce6dd7e49b6410d1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 450kB 42.9MB/s 
     |████████████████████████████████| 3.9MB 42.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=62311d1e6a2ea909f541c5a8822c30f983180f73bfaa800823321f4fc982dca4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.

In [0]:
import tensorflow
import torch, torchvision
import torch.nn as nn
import time
import random
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()

import transformers
from transformers import XLMRobertaModel, XLMRobertaTokenizer, XLMRobertaConfig, AutoTokenizer, TFAutoModel, AutoModelWithLMHead
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule

import re

In [0]:
cd /content/gdrive/My Drive

[Errno 2] No such file or directory: '/content/gdrive/My Drive'
/content


In [0]:
import pandas as pd
import os

data_path = '/content/gdrive/My Drive/Kaggle/'

# TEST_PATH = os.path.join(data_path, "test.csv")
# VAL_PATH = os.path.join(data_path, "validation.csv")
# TRAIN_PATH = os.path.join(data_path, "jigsaw-toxic-comment-train.csv")

# val_data = pd.read_csv(VAL_PATH)
# test_data = pd.read_csv(TEST_PATH)
# train_data = pd.read_csv(TRAIN_PATH)

# sub = pd.read_csv('/content/gdrive/My Drive/Kaggle/sample_submission.csv')

train1 = pd.read_csv("/content/drive/My Drive/Kaggle/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/content/drive/My Drive/Kaggle/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/content/drive/My Drive/Kaggle/validation.csv')
test = pd.read_csv('/content/drive/My Drive/Kaggle/test.csv')
sub = pd.read_csv('/content/drive/My Drive/Kaggle/sample_submission.csv')

# Combine train1 with a subset of train2
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

In [0]:
MAX_LEN = 512
MODEL = 'xlm-roberta-large'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL, do_lower_case=True)
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
device = "cuda"


class BERTDataset:
    def __init__(self, comment_text, target):
        self.comment_text = comment_text
        self.target = target
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, item):
        comment_text = str(self.comment_text[item])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.target[item], dtype=torch.float)
        }

In [0]:
train_dataset = BERTDataset(comment_text=train.comment_text.values, target=train.toxic.values)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

valid_dataset = BERTDataset(comment_text=valid.comment_text.values, target=valid.toxic.values)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

In [0]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = AutoModelWithLMHead.from_pretrained(MODEL)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768*2, 1)

    def forward(self, ids, mask, token_type_ids):
        o1, _ = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        mean_pooling = torch.mean(o1, 1)
        max_pooling, _ = torch.max(o1, 1)
        cat = torch.cat((mean_pooling, max_pooling), 1)


        bo = self.bert_drop(cat)

        output = self.out(bo)

        return output

In [0]:
model = BERTBaseUncased()


In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]

In [0]:
    num_train_steps = int(len(train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    model = nn.DataParallel(model)

In [0]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [0]:
best_accuracy = 0
for epoch in range(EPOCHS):
  train_fn(train_data_loader, model, optimizer, device, scheduler)
  outputs, targets = eval_fn(valid_data_loader, model, device)
  targets = np.array(targets) >= 0.5
  accuracy = metrics.roc_auc_score(targets, outputs)
  print(f"AUC Score = {accuracy}")
  if accuracy > best_accuracy:
    torch.save(model.state_dict(), config.MODEL_PATH)
    best_accuracy = accuracy

  0%|          | 0/54472 [00:00<?, ?it/s]

KeyError: ignored